# Introduction 

In this Lab, we are going to introduce the utilization of several statistical methods in AB testing (mainly about hypothesis testing and ANOVA). The dataset is provided by Breza E et al.[1]. Through this dataset, we want to figure out whether the public health messages sent by professionals can really influence people's behaviour during the COVID-19 crisis.

#### Notes:
The raw dataset provided by the article is very complex, and the data cleaning procedures are complicated. Thus, here we omit the preprocess procedures and provide you a dataset that have already been modified. If you are intersting in more informations, you can download the original dataset and corresponding R-code from the website given by the authors in the artile. 

#### Referrence:
[1] Breza E, Stanford F C, Alsan M, et al. Effects of a large-scale social media advertising campaign on holiday travel and COVID-19
infections: a cluster randomized controlled trial[J]. *Nature medicine*, 2021, 27(9): 1622-1628.

In [1]:
# import the necessary library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

In [2]:
# load the dataset
data = pd.read_csv('data&analysis.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6024 entries, 0 to 6023
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_loc         6024 non-null   int64 
 1   zip              6024 non-null   int64 
 2   state            6024 non-null   object
 3   high_county_T1   6024 non-null   int64 
 4   treated_T1       6024 non-null   int64 
 5   urban            6024 non-null   int64 
 6   two_weeks_cases  6024 non-null   int64 
 7   majority_gop     6024 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 376.6+ KB


In [3]:
data.head()

,user_loc,zip,state,high_county_T1,treated_T1,urban,two_weeks_cases,majority_gop
0,4001,85925,AZ,0,0,1,0,0
1,4001,85924,AZ,0,0,0,21,0
2,4001,85936,AZ,0,0,1,14,0
3,4001,85940,AZ,0,0,0,0,0
4,4001,86505,AZ,0,0,0,11,0


In [4]:
data.loc[data['high_county_T1']==0].describe()

,user_loc,zip,high_county_T1,treated_T1,urban,two_weeks_cases,majority_gop
count,2910.000000,2910.000000,2910.0,2910.000000,2910.000000,2910.000000,2910.000000
mean,24848.143643,48236.434708,0.0,0.249828,0.555326,92.438832,0.674227
std,14723.517922,22235.212970,0.0,0.432988,0.497015,199.255925,0.468744
min,4001.000000,2802.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,12103.000000,27959.750000,0.0,0.000000,0.000000,4.000000,0.000000
50%,18179.000000,47849.000000,0.0,0.000000,1.000000,22.000000,1.000000
75%,37158.500000,62659.500000,0.0,0.000000,1.000000,93.000000,1.000000
max,51810.000000,97630.000000,0.0,1.000000,1.000000,2744.000000,1.000000


In [5]:
data.loc[data['high_county_T1']==1].describe()

,user_loc,zip,high_county_T1,treated_T1,urban,two_weeks_cases,majority_gop
count,3114.000000,3114.000000,3114.0,3114.000000,3114.000000,3114.000000,3114.000000
mean,26146.884393,48330.790623,1.0,0.745022,0.578356,90.239563,0.656069
std,13818.149251,22095.737032,0.0,0.435919,0.493902,172.544960,0.475095
min,4005.000000,2804.000000,1.0,0.000000,0.000000,0.000000,0.000000
25%,17031.000000,28450.250000,1.0,0.000000,0.000000,5.000000,0.000000
50%,24031.000000,47959.500000,1.0,1.000000,1.000000,24.000000,1.000000
75%,39300.500000,62546.750000,1.0,1.000000,1.000000,98.000000,1.000000
max,51840.000000,97701.000000,1.0,1.000000,1.000000,2026.000000,1.000000


## 1. Hypothesis Testing

In this section, we are going to deliver the upper-tailed test to study whether the population mean of treatment group is less than the control group. 

In [6]:
treatment = data.loc[data['treated_T1']==1]
control = data.loc[data['treated_T1']==0]
treatment_low = treatment.loc[treatment['high_county_T1']==0]
control_low = control.loc[control['high_county_T1']==0]
treatment_high = treatment.loc[treatment['high_county_T1']==1]
control_high = control.loc[control['high_county_T1']==1]

In [7]:
treatment['two_weeks_cases'].describe()
# treatment_low['two_weeks_cases'].describe()
# treatment_high['two_weeks_cases'].describe()

count    3047.000000
mean       89.700033
std       177.337531
min         0.000000
25%         4.000000
50%        22.000000
75%       100.000000
max      2744.000000
Name: two_weeks_cases, dtype: float64

In [8]:
control['two_weeks_cases'].describe()
# control_low['two_weeks_cases'].describe()
# control_high['two_weeks_cases'].describe()

count    2977.000000
mean       92.941552
std       194.319811
min         0.000000
25%         4.000000
50%        24.000000
75%        94.000000
max      2702.000000
Name: two_weeks_cases, dtype: float64

In [12]:

'''
H_0: the average COVID-19 infections in areas with intervention is greater or equal to that in the areas without intervention
H_a: the average COVID-19 infections in areas with intervention is less than that in the areas without intervention
'''
#  Notice that the p-value output by function 'scipy.stats.ttest_ind' is two-tailed probability
t, pval = stats.ttest_ind(treatment['two_weeks_cases'], control['two_weeks_cases'], equal_var=False)
print('t-statistic:', t, 'p-value:', pval / 2)
print('p > 0.05, we cannot reject H_0')

t-statistic: -0.6758279130183211 p-value: 0.2495881006264814
p > 0.05, we cannot reject H_0


In [13]:
'''
H_0: In low-intensity counties, the average COVID-19 infections in areas with intervention is greater or equal to that in the areas without intervention
H_a: In low-intensity counties, the average COVID-19 infections in areas with intervention is less than that in the areas without intervention
'''
t_low, pval_low = stats.ttest_ind(treatment_low['two_weeks_cases'], control_low['two_weeks_cases'], equal_var=False)
print('t-statistic:', t_low, 'p-value:', pval_low / 2)
print('p > 0.05, we cannot reject H_0')

t-statistic: 0.8595339069226191 p-value: 0.19511308178658343
p > 0.05, we cannot reject H_0


In [14]:
'''
H_0: In high-intensity counties, the average COVID-19 infections in areas with intervention is greater or equal to that in the areas without intervention
H_a: In high-intensity counties, the average COVID-19 infections in areas with intervention is less than that in the areas without intervention
'''
t_high, pval_high = stats.ttest_ind(treatment_high['two_weeks_cases'], control_high['two_weeks_cases'], equal_var=False)
print('t-statistic:', t_high, 'p-value:', pval_high / 2)
print('p > 0.05, we cannot reject H_0')

'''But if we select the level of significance as 0.1, then we may conclude that p < 0.1 and we can reject H_0. That's to say, it's considered that the intervention decreases the COVID-19 infections in high-intensity counties.'''

t-statistic: -1.628760833096124 p-value: 0.05181373951793404
p > 0.05, we cannot reject H_0


"But if we select the level of significance as 0.1, then we may conclude that p < 0.1 and we can reject H_0. That's to say, it's considered that the intervention decreases the COVID-19 infections in high-intensity counties."

## 2. Two way ANOVA

In this section, we are going to two-way ANOVA to study whether the intervention has the same impact on counties with different statuses. 

In [16]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

formula = 'two_weeks_cases ~ C(treated_T1) + C(urban) + C(treated_T1) : C(urban)'
data_low = data.loc[data['high_county_T1']==0]
model_low = ols(formula, data_low).fit()
anova_table = anova_lm(model_low, type=2)
anova_table

,df,sum_sq,mean_sq,F,PR(>F)
C(treated_T1),1.0,3.256024e+04,3.256024e+04,0.928022,3.354575e-01
C(urban),1.0,1.345125e+07,1.345125e+07,383.383722,2.735060e-80
C(treated_T1):C(urban),1.0,5.319260e+04,5.319260e+04,1.516080,2.183133e-01
Residual,2906.0,1.019588e+08,3.508562e+04,NaN,NaN


In [17]:
data_high = data.loc[data['high_county_T1']==1]
model_high = ols(formula, data_high).fit()
anova_table = anova_lm(model_high, type=2)
anova_table

,df,sum_sq,mean_sq,F,PR(>F)
C(treated_T1),1.0,9.399691e+04,9.399691e+04,3.646061,5.629390e-02
C(urban),1.0,1.234389e+07,1.234389e+07,478.809044,7.702414e-99
C(treated_T1):C(urban),1.0,6.457622e+04,6.457622e+04,2.504857,1.135974e-01
Residual,3110.0,8.017704e+07,2.578040e+04,NaN,NaN
